In [1]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

In [2]:
f = Foo()

In [3]:
f[1]

10

In [4]:
for i in f:
    print(i)

0
10
20


In [5]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck():
    rank = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds club hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank ,suit) for suit in self.suits
                                        for rank in self.rank]
    
    def __getitem__(self, item):
        return self._cards[item]
    
    def __len__(self):
        return len(self._cards)

In [7]:
from random import shuffle
l = list(range(10))
shuffle(l)
l

[3, 4, 0, 8, 2, 7, 5, 6, 9, 1]

In [8]:
deck = FrenchDeck()

In [9]:
shuffle(deck) 
# 'FrenchDeck' object does not support item assignment说明FrenchDeck只实现了不可变的协议序列
# 可变序列必须提供__setitem__用法

TypeError: 'FrenchDeck' object does not support item assignment

In [10]:
# 因为python是动态语言，可以在程序运行时修正这个问题

In [12]:
def set_card(deck, pos, card):
    deck._cards[pos] = card

In [15]:
FrenchDeck.__setitem__ = set_card

In [17]:
shuffle(deck)
deck[:5]

[Card(rank='9', suit='hearts'),
 Card(rank='Q', suit='club'),
 Card(rank='Q', suit='spades'),
 Card(rank='A', suit='club'),
 Card(rank='6', suit='hearts')]

In [18]:
# 抽象基类
# 二、抽象基类
# 1、基本概念
# 鸭子类型(duck typing)：不关注对象的类型，而是关注对象具有的行为(方法)。鸭子类型像多态一样工作，但是没有继承。
# 在鸭子类型中，协议风格的接口与继承完全没有关系，实现同一个协议的各个类是相互独立的。

# 白鹅类型(goose typing)：只要cls是抽象基类，即cls的元类是abc.ABCMeta,就可以使用isinstance(obj,cls)。
# 抽象基类（abstract base class,ABC）：抽象基类就是类里定义了纯虚成员函数的类。纯虚函数只提供了接口，
# 并没有具体实现。抽象基类不能被实例化(不能创建对象)，通常是作为基类供子类继承，子类中重写虚函数，实现具体的接口。
# 简言之，ABC描述的是至少使用一个纯虚函数的接口，从ABC派生出的类将根据派生类的具体特征，使用常规虚函数来实现这种接口。

In [19]:
# 定义一个抽象基类

In [22]:
import abc

class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        '''从可迭代对象中添加元素'''

    @abc.abstractmethod
    def pick(self):
        '''
        随机删除元素，然后将其返回
        如果实例为空，这个方法应该抛出"LookupError"
        '''

    def loaded(self):
        '''
        判断是否还有元素存在
        '''
        return bool(self.inspect())

    def inspect(self):
        """返回一个有序元组，有当前元素组成"""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [25]:
class Fake(Tombola):
    def pick(self):
        return 13

In [26]:
#实例化Fake必须实现抽象基类的 load 和 pick方法

In [27]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

In [28]:
import random

In [29]:
class LotteryBlower(Tombola):

    def __init__(self, iteriable):
        self._balls = list(iteriable)

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            pos = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LotteryBlower')
        return self._balls.pop(pos)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))

In [31]:
a = LotteryBlower([1,2,2]) # 实例化成功

In [32]:
#不用继承，也可将一个类注册为Tombola的虚拟子类

In [33]:
@Tombola.register  # 注册为 Tombola 的虚拟子类。
class TomboList(list):  # 扩展 list

    def pick(self):
        if self:  # 从 list 中继承 __bool__ 方法，列表不为空时返回 True。
            position = random.randrange(len(self))
            return self.pop(position)  # 调用继承自 list 的 self.pop 方法，传入一个随机的元素索引。
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend  # 与 list.extend 一样

    def loaded(self):
        return bool(self)  # 委托 bool 函数。

    def inspect(self):
        return tuple(sorted(self))

In [34]:
issubclass(TomboList, Tombola)

True

In [35]:
t = TomboList(range(100))

In [37]:
isinstance(t, Tombola)

True

In [38]:
TomboList.__mro__# 查看超类

(__main__.TomboList, list, object)

In [39]:
# 发现TomboList 未继承Tombola

In [40]:
Tombola.__subclasses__()

[__main__.Fake, __main__.LotteryBlower]

In [42]:
Tombola._abc_registry

In [50]:
class s():
    def __len__():
        return 23

In [58]:
'''
__subclasshook__特殊方法.
可以完全使用不相关的类，只要实现特定的方法即可, 当然，只有提供 __subclasshook__ 方法的 抽象类 才能这么做。
在自己定义的抽象基类中要不要实现 __subclasshook__ 方法呢？可能不需要。
'''
def _check_methods(C, *methods):
    mro = C.__mro__
    for method in methods:
        for B in mro:
            if method in B.__dict__:
                if B.__dict__[method] is None:
                    return NotImplemented
                break
        else:
            return NotImplemented
    return True

class Sized(metaclass=abc.ABCMeta):

    __slots__ = ()   # 防止被动态赋予属性？

    @abc.abstractmethod
    def __len__(self):
        return 0

    @classmethod
    def __subclasshook__(cls, C):  # C 是否为 cls 的子类.
        if cls is Sized:
            return _check_methods(C, "__len__")  # 表明 C 是否为 cls 的子类.
        return NotImplemented  # 返回 让子类检查。

In [59]:
issubclass(s, Sized)

True

In [60]:
isinstance(s(), Sized)

True